In [ ]:
'''
将原始验证码图片处理成去除背景的图片
'''

import cv2

def check_img(path):
    image = cv2.imread(path)
    print(image.shape)
    
    rbg_1st_largest_value = 210
    rbg_2nd_largest_value = 150
    rbg_3nd_largest_value = 100
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            fst_count = 0
            snd_count = 0
            trd_count = 0
            if image[i,j,0]>rbg_3nd_largest_value:
                if image[i,j,0]>rbg_2nd_largest_value:
                    if image[i,j,0]>rbg_1st_largest_value:
                        fst_count = fst_count+1
                    else:
                        snd_count = snd_count+1
                else:
                    trd_count = trd_count+1
            if image[i,j,1]>rbg_3nd_largest_value:
                if image[i,j,1]>rbg_2nd_largest_value:
                    if image[i,j,1]>rbg_1st_largest_value:
                        fst_count = fst_count+1
                    else:
                        snd_count = snd_count+1
                else:
                    trd_count = trd_count+1
            if image[i,j,2]>rbg_3nd_largest_value:
                if image[i,j,2]>rbg_2nd_largest_value:
                    if image[i,j,2]>rbg_1st_largest_value:
                        fst_count = fst_count+1
                    else:
                        snd_count = snd_count+1
                else:
                    trd_count = trd_count+1
            if fst_count==0:
                image[i,j,:] = [0, 0, 0]
            if fst_count+snd_count==3:
                image[i,j,:] = [0, 0, 0]
            # if (fst_count+snd_count==2) and trd_count==1:
            #     image[i,j,:] = [0, 0, 0]

    for i in range(320):
        print(image[0,i,:])
    
    cv2.imwrite(".\\tmp.jpg", image)

if 1:
    path = r"D:\vscode_workspace\Python\ZhongYi\chaoxin\4.jpg"
    check_img(path)

In [ ]:
'''
字体
'''

from PIL import Image,ImageDraw,ImageFont
name = r"D:\vscode_workspace\Python\ZhongYi\chaoxin\7.jpg"
img = Image.open(name)
font = ImageFont.truetype(r"D:\vscode_workspace\Python\ZhongYi\chaoxin\msyhl.ttc", 40, encoding="unic")
draw = ImageDraw.Draw(img)
cc = "你好"
draw.text((20, 20), str(cc), fill=(0, 150, 160), font=font)
img.save(name)

In [7]:
'''
灰度化
'''

from PIL import Image
img_path = r"D:\vscode_workspace\Python\ZhongYi\chaoxin\1.jpg"
image = Image.open(img_path)
img_pixel = image.load()

threshold = 200
# for x in range(image.size[0]):
#     for y in range(image.size[1]):
#         if img_pixel[x, y][0]>=threshold or img_pixel[x, y][1]>=threshold or img_pixel[x, y][2]>=threshold:
#             pass
#         else:
#             img_pixel[x, y] = (0, 0, 0)

image = image.convert('L')
image.save(r"D:\vscode_workspace\Python\ZhongYi\chaoxin\tmp_L.jpg")


In [ ]:
'''生成渐变色'''

import numpy as np
from PIL import Image

def RGB(r,g,b): return (r,g,b)

def Make_gradation_img_data(width, height, rgb_start, rgb_stop, horizontal=(True, True, True)):
    '''Make gradation image data'''
    result = np.zeros((height, width, 3), dtype=np.uint8)
    for i, (m,n,o) in enumerate(zip(rgb_start, rgb_stop, horizontal)):
        if o:
            result[:,:,i] = np.tile(np.linspace(m, n, width), (height, 1))
        else:
            result[:,:,i] = np.tile(np.linspace(m, n, width), (height, 1)).T
    return result

MakeGradationImg = lambda width, height, rgb_start, rgb_stop, horizontal=(True, True, True):Image.fromarray(Make_gradation_img_data(width, height, rgb_start, rgb_stop, horizontal))

img = MakeGradationImg(40, 40, RGB(255,180,0), RGB(0,255,180), (True, True, True))
img.save('img_001.png')


In [11]:
import json, random
from PIL import Image,ImageDraw,ImageFont
import numpy as np

def generate_color():
    '''生成高亮度的颜色'''
    tmp = [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]
    index = random.randint(0, 2)
    tmp[index] = random.randint(253, 255)
    tmp[(index+random.randint(1, 2))%3] = random.randint(1, 10)
    return tuple(tmp)

def color(img):
    '''给汉字上色（渐变色）'''
    # 1/5概率不使用渐变色，直接退出函数
    if not random.randint(0, 5):
        return img
    def Make_gradation_img_data(width, height, rgb_start, rgb_stop, horizontal=(True, True, True)):
        '''Make gradation image data'''
        result = np.zeros((height, width, 3), dtype=np.uint8)
        for i, (m,n,o) in enumerate(zip(rgb_start, rgb_stop, horizontal)):
            if o:
                result[:,:,i] = np.tile(np.linspace(m, n, width), (height, 1))
            else:
                result[:,:,i] = np.tile(np.linspace(m, n, width), (height, 1)).T
        return result

    MakeGradationImg = lambda width, height, rgb_start, rgb_stop, horizontal=(True, True, True):Image.fromarray(Make_gradation_img_data(width, height, rgb_start, rgb_stop, horizontal))

    # 生成渐变色图片
    raw_size = 60
    gra_img = MakeGradationImg(raw_size, raw_size, generate_color(), generate_color(), (True, True, True))
    gra_img = gra_img.rotate(45)
    box = ((raw_size-40)/2, (raw_size-40)/2, (raw_size-40)/2+40, (raw_size-40)/2+40)
    gra_img = gra_img.crop(box)
    gra_img = gra_img.convert('RGBA')

    # 抠出汉字
    gra_pixel = gra_img.load()
    img_pixel = img.load()
    for x in range(img.size[0]):
        for y in range(img.size[1]):
            if img_pixel[x, y][3]!=0:
                gra_pixel[x, y] = (gra_pixel[x, y][0], gra_pixel[x, y][1], gra_pixel[x, y][2], 255)
            else:
                gra_pixel[x, y] = (gra_pixel[x, y][0], gra_pixel[x, y][1], gra_pixel[x, y][2], 0)

    return gra_img
    
def mess(img):
    '''给图像加上噪点'''
    img_pixels = img.load()
    noise = 20
    for x in range(img.size[0]):
        for y in range(img.size[1]):
            if img_pixels[x, y][3]!=0:
                img_pixels[x, y] = (img_pixels[x, y][0]+random.randint(-noise, noise), img_pixels[x, y][1]+random.randint(-noise, noise), img_pixels[x, y][2]+random.randint(-noise, noise), 255)
    return img

def shadow(hanzi_img):
    '''向左上方投影1像素'''
    shadow = Image.new('RGBA', hanzi_img.size, (0, 0, 0, 0))
    shift_pos = 1
    hanziz_pixels = hanzi_img.load()
    shadow_pixels = shadow.load()
    for x in range(hanzi_img.size[0]):
        for y in range(hanzi_img.size[1]):
            if hanziz_pixels[x, y][3]!=0:
                if x>=shift_pos and y>=shift_pos:
                    shadow_pixels[x-shift_pos, y-shift_pos] = (int(hanziz_pixels[x, y][0]/4), int(hanziz_pixels[x, y][1]/4), int(hanziz_pixels[x, y][2]/4), 255)
    
    shadow.paste(hanzi_img, (0, 0), mask=hanzi_img)
    return shadow

def append_hanzi(raw_img, character):
    '''选取五个汉字中的一个，将其放在图片右侧'''
    out_img = Image.new('L', (360, 160), (0))
    draw = ImageDraw.Draw(out_img)
    draw.text((320, 0), character, fill=(255), font=ImageFont.truetype(r"D:\vscode_workspace\Python\ZhongYi\chaoxin\ttc\msyh.ttc", 28, encoding="unic"))
    draw.text((320, 40), character, fill=(255), font=ImageFont.truetype(r"D:\vscode_workspace\Python\ZhongYi\chaoxin\ttc\STXINGKA.TTF", 40, encoding="unic"))
    draw.text((320, 80), character, fill=(255), font=ImageFont.truetype(r"D:\vscode_workspace\Python\ZhongYi\chaoxin\ttc\STZHONGS.TTF", 30, encoding="unic"))
    draw.text((320, 120), character, fill=(255), font=ImageFont.truetype(r"D:\vscode_workspace\Python\ZhongYi\chaoxin\ttc\SIMLI.TTF", 30, encoding="unic"))
    out_img.paste(raw_img, (0, 0), mask=raw_img)
    return out_img

def generate_pic(output_index):
    # 读取常用汉字列表
    hanzi_path = r"D:\vscode_workspace\Python\ZhongYi\chaoxin\common_chinese_characters.json"
    with open(hanzi_path, 'r', encoding='utf-8') as f:
        hanzi_list = json.load(f)

    # 读取不同字体
    fonts = []
    fonts.append(ImageFont.truetype(r"D:\vscode_workspace\Python\ZhongYi\chaoxin\ttc\msyh.ttc", 28, encoding="unic"))
    fonts.append(ImageFont.truetype(r"D:\vscode_workspace\Python\ZhongYi\chaoxin\ttc\STXINGKA.TTF", 40, encoding="unic"))
    fonts.append(ImageFont.truetype(r"D:\vscode_workspace\Python\ZhongYi\chaoxin\ttc\STZHONGS.TTF", 30, encoding="unic"))

    # 随机读取一张动物图片
    animal_path = r"D:\vscode_workspace\Python\ZhongYi\chaoxin\reshaped\\"+str(random.randint(1,501))+".jpg"
    full_img = Image.open(animal_path)
    full_img = full_img.convert('RGBA')

    # 添加汉字
    characters = []
    character_fonts = [] # 字体
    character_rotates = [] # 旋转度数
    character_poses = [] # 位置
    for i in range(5):
        img = Image.new('RGBA', (40, 40), (0, 0, 0, 0))
        # 生成随机汉字
        character = hanzi_list[random.randint(0, len(hanzi_list)-1)]
        characters.append(character)
        # 随机选择字体
        character_fonts.append(random.randint(0, len(fonts)-1))
        # 绘制汉字
        draw = ImageDraw.Draw(img)
        draw.text((0, 0), characters[i], fill=generate_color(), font=fonts[character_fonts[i]])
        # 上色
        img = color(img)
        # 旋转汉字
        character_rotates.append(random.randint(-45, 45))
        img = img.rotate(character_rotates[i])
        # 阴影(阴影要放在旋转之前，让旋转操作让阴影变得更自然（相当于额外平滑了一次像素）)
        img = shadow(img)
        # 加噪点
        img = mess(img)
        # 覆盖图片
        label = True
        tmp = (0, 0)
        while label:
            label = False
            tmp = (random.randint(0, 280), random.randint(0, 120))
            for item in character_poses:
                if tmp[0]-item[0]>= -25 and tmp[0]-item[0]<=25 and tmp[1]-item[1]>= -25 and tmp[1]-item[1]<=25:
                    label = True
        character_poses.append(tmp)
        full_img.paste(img, character_poses[i], mask=img)
    # 灰度化
    full_img = full_img.convert('L')
    # 选取五个汉字中的一个，将其放在图片右侧
    which_hanzi = random.randint(0, 4)
    full_img = append_hanzi(full_img, characters[which_hanzi])
    full_img.save(r"D:\vscode_workspace\Python\ZhongYi\chaoxin\tmp.jpg")


if 1:
    generate_pic(1)

In [ ]:
'''
旋转
'''

from PIL import Image

img = Image.open("Koala.jpg")

# 旋转方式一
img1 = img.transpose(Image.ROTATE_180)   # 引用固定的常量值
img1.save("r1.jpg")

# 旋转方式二
img2 = img.rotate(90)   # 自定义旋转度数
img2 = img2.resize((400, 400))   # 改变图片尺寸
img2.save("r2.jpg")

In [ ]:
'''
裁剪
'''

import random, cv2
'''本部分是将 https://photo.goldposter.com/zh/tag/animals/ 上下载的图片裁剪成符合要求的格式'''
def reshape(img_path, i):
    image = cv2.imread(img_path)
    if image.shape[1]<320 or image.shape[0]<160:
        print(str(i), "'s shape doesn't satisfy:", image.shape)
        return
    else:
        
        print(str(i), "'s shape:", image.shape)
    x_start = random.randint(0, image.shape[1]-320)
    y_start = random.randint(0, image.shape[0]-160)
    cropped = image[y_start:y_start+160, x_start:x_start+320, :]  # 裁剪坐标为[y0:y1, x0:x1]
    write_path = r"D:\vscode_workspace\Python\ZhongYi\chaoxin\reshaped" + "\\" + str(i) + ".jpg"
    cv2.imwrite(write_path, cropped)

if 1:
    for i in range(501):
        try:
            if i+1<10:
                path = r"D:\vscode_workspace\Python\ZhongYi\chaoxin\animals_GoldPoster_Free_Stock_Photos\1000" + str(i+1) + ".jpg"
            elif i+1<100:
                path = r"D:\vscode_workspace\Python\ZhongYi\chaoxin\animals_GoldPoster_Free_Stock_Photos\100" + str(i+1) + ".jpg"
            elif i+1<1000:
                path = r"D:\vscode_workspace\Python\ZhongYi\chaoxin\animals_GoldPoster_Free_Stock_Photos\10" + str(i+1) + ".jpg"
            print(path)
            reshape(path, i+1)
        except:
            if i+1<10:
                path = r"D:\vscode_workspace\Python\ZhongYi\chaoxin\animals_GoldPoster_Free_Stock_Photos\1000" + str(i+1) + ".png"
            elif i+1<100:
                path = r"D:\vscode_workspace\Python\ZhongYi\chaoxin\animals_GoldPoster_Free_Stock_Photos\100" + str(i+1) + ".png"
            elif i+1<1000:
                path = r"D:\vscode_workspace\Python\ZhongYi\chaoxin\animals_GoldPoster_Free_Stock_Photos\10" + str(i+1) + ".png"
            print(path)
            reshape(path, i+1)

In [ ]:
'''
生成常用的汉字json
'''

import json

characters = [] # 创建一个列表用于保存汉字字符
for i in range(176, 216):
    s = bytes([i])
    for x in range(161, 255):
        s += bytes([x])
        try:
            c = s.decode("gb2312")
        except:
            break
        characters.append(c)
        print(c, end="\t") # 打印结果
        s = bytes([i])

print(len(characters)) # 打印结果数量

filename = "common_chinese_characters.json"
with open(filename, "w", encoding="utf-8") as f:
    json.dump(characters, f, ensure_ascii=False)

